# Camada Silver - Data Quality e Conformidade

---

## Objetivo
A Camada Silver é o núcleo de higienização e conformidade do nosso Data Lakehouse. Este notebook consome os dados brutos da Camada Bronze e aplica um **Motor Universal de Data Quality (DQ)**. O objetivo é transformar textos sujos e dados legados em informações tipadas, confiáveis e prontas para cálculos financeiros ou modelagem de Machine Learning.

## Lógica do Motor de Data Quality (DQ)
O pipeline não utiliza *hardcode* de colunas. Ele lê os metadados dinamicamente e aplica 3 regras universais:
1. **Limpeza Financeira (`DecimalType`):** Identifica colunas de moedas (sufixo `_R$` ou `_U$`), remove os pontos de milhar, substitui a vírgula decimal por ponto e aplica o casting seguro para `Decimal(18,2)`.
2. **Time Travel Parsing (`DateType`):** Identifica colunas temporais (prefixo `DATA_`), higieniza strings anômalas (ex: "Não informada") e converte o padrão de texto brasileiro (`dd/MM/yyyy`) para o formato nativo de datas.
3. **Extermínio de Magic Strings (Nullification):** Sistemas governamentais preenchem lacunas com valores fictícios. O script possui um dicionário de anomalias conhecidas (`["-1", "-3", "Sem informação", "Inválido", ""]`) e as converte massivamente para verdadeiros `NULL`s do banco de dados, evitando agrupamentos fantasmas nos modelos analíticos.

## Decisões de Engenharia
* **Schema Evolution:** A transição de tipos primitivos (de `StringType` na Bronze para `DecimalType` e `DateType` na Silver) aciona a trava de segurança nativa do Delta Lake (*Schema Enforcement*). Para permitir essa alteração de contrato de forma gerida, utilizamos a *flag* `.option("overwriteSchema", "true")`.
* **Particionamento:** A arquitetura mantém a partição nativa por `posicao_base`, essencial para a performance das queries do time de negócios.

---
**Origem:** Tabelas gerenciadas Delta (`public_informations.bronze_...`)

**Destino:** Tabelas tipadas e limpas (`public_informations.silver_...`)

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import DecimalType
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

SCHEMA_NAME = "public_informations"
DOMAINS = ["cadastro", "remuneracao", "afastamentos", "observacoes"]

# Lista de lixos conhecidos do SIAPE
VALORES_ANOMALOS = ["-1", "-3", "Sem informação", "Sem informaç", "null", "NULL", "Inválido", "Não informada", ""]

def transform_to_silver_with_dq():
    for domain in DOMAINS:
        logger.info(f"Iniciando transformação Silver c/ DQ para o domínio: {domain.upper()}")
        
        bronze_table = f"{SCHEMA_NAME}.bronze_{domain}"
        silver_table = f"{SCHEMA_NAME}.silver_{domain}"
        
        try:
            df = spark.read.table(bronze_table)
            
            for col in ["ANO", "MES"]:
                if col in df.columns:
                    df = df.drop(col)
            
            # --- MOTOR DE DATA QUALITY UNIVERSAL ---
            for col_name in df.columns:
                
                # Converte R$/U$ para Decimal(18,2)
                if "_R$" in col_name or "_U$" in col_name:
                    df = (df
                          .withColumn(col_name, F.regexp_replace(F.col(col_name), r'\.', ''))
                          .withColumn(col_name, F.regexp_replace(F.col(col_name), r'\,', '.'))
                          .withColumn(col_name, F.col(col_name).cast(DecimalType(18, 2))))
                
                # Converte string 'dd/MM/yyyy' para DateType
                elif col_name.startswith("DATA_"):
                    # Primeiro, limpa strings estranhas como "Não informada", depois converte para data
                    df = df.withColumn(col_name, F.when(F.col(col_name).isin(VALORES_ANOMALOS), F.lit(None))
                                                  .otherwise(F.col(col_name)))
                    df = df.withColumn(col_name, F.to_date(F.col(col_name), "dd/MM/yyyy"))
                
                # Transforma lixo em NULL
                # Aplica apenas em colunas de string (que não sejam arquivo/posicao)
                elif dict(df.dtypes)[col_name] == 'string' and col_name not in ["arquivo_origem", "posicao_base"]:
                    df = df.withColumn(col_name, F.when(F.col(col_name).isin(VALORES_ANOMALOS), F.lit(None))
                                                  .otherwise(F.col(col_name)))

            # Adição de Metadados da Silver
            df_silver = df.withColumn("timestamp_carga_silver", F.current_timestamp())
            
            # Salva a Tabela Gerenciada na Camada Silver
            (df_silver.write
             .format("delta")
             .mode("overwrite")
             .option("overwriteSchema", "true")
             .partitionBy("posicao_base")
             .saveAsTable(silver_table))
            
            logger.info(f" -> ✅ Tabela {silver_table} processada, limpa e catalogada com sucesso!")
            
        except Exception as e:
            logger.error(f" -> ❌ Erro ao processar a tabela {domain.upper()}: {e}")

if __name__ == "__main__":
    logger.info("Iniciando pipeline Avançado da Camada Silver...")
    transform_to_silver_with_dq()
    logger.info("Pipeline Silver finalizado com sucesso!")

2026-02-25 22:02:41,715 - Iniciando pipeline Avançado da Camada Silver...
2026-02-25 22:02:41,718 - Iniciando transformação Silver c/ DQ para o domínio: CADASTRO
2026-02-25 22:02:56,085 -  -> ✅ Tabela public_informations.silver_cadastro processada, limpa e catalogada com sucesso!
2026-02-25 22:02:56,085 - Iniciando transformação Silver c/ DQ para o domínio: REMUNERACAO
2026-02-25 22:03:09,820 -  -> ✅ Tabela public_informations.silver_remuneracao processada, limpa e catalogada com sucesso!
2026-02-25 22:03:09,821 - Iniciando transformação Silver c/ DQ para o domínio: AFASTAMENTOS
2026-02-25 22:03:13,390 -  -> ✅ Tabela public_informations.silver_afastamentos processada, limpa e catalogada com sucesso!
2026-02-25 22:03:13,391 - Iniciando transformação Silver c/ DQ para o domínio: OBSERVACOES
2026-02-25 22:03:17,136 -  -> ✅ Tabela public_informations.silver_observacoes processada, limpa e catalogada com sucesso!
2026-02-25 22:03:17,137 - Pipeline Silver finalizado com sucesso!
